In [1]:
%cd ../../../../

/home/users/dmoreno2016/ASTROMER/astromer_pe_main/astromer


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

import os

from src.data.record import deserialize
from src.data.zero import to_windows

%load_ext autoreload
%autoreload 2

2023-11-06 05:06:18.119681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-06 05:06:19.458329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def get_dataset(path_dataset):
    rec_paths = []
    for folder in os.listdir(path_dataset):
        if folder.endswith('.csv'):
            continue
        for x in os.listdir(os.path.join(path_dataset, folder)):
            rec_paths.append(os.path.join(path_dataset, folder, x))

    dataset = tf.data.TFRecordDataset(rec_paths)    
    dataset = dataset.map(deserialize)

    return dataset

def get_windows(dataset, window_size, sampling):
    
    dataset = to_windows(dataset,
                        window_size=window_size,
                        sampling=sampling)

    return dataset

In [4]:
# Macho unlabeled
dataset_macho_unl = get_dataset(path_dataset='./data/records/macho_clean/train')

# Macho labeled
dataset_macho_lab = get_dataset(path_dataset='./data/records/alcock/fold_0/alcock_50/train')

# Atlas
dataset_atlas = get_dataset(path_dataset='./data/records/atlas/fold_0/atlas_50/train')

# Ogle
dataset_ogle = get_dataset(path_dataset='./data/records/ogle/fold_0/ogle_50/train')

# Kepler
dataset_kepler = get_dataset(path_dataset='./data/records/kepler/fold_0/kepler_50/train')
dataset_kepler_alcock = get_dataset(path_dataset='./data/records/kepler_alcock_linear/fold_0/kepler_alcock_linear_50/train')
dataset_kepler_atlas = get_dataset(path_dataset='./data/records/kepler_atlas_linear/fold_0/kepler_atlas_linear_50/train')
dataset_kepler_ogle = get_dataset(path_dataset='./data/records/kepler_ogle_linear/fold_0/kepler_ogle_linear_50/train')

2023-11-06 05:06:24.715316: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
# CREATE WINDOWS
window_size = 200
sampling = False

# Macho unlabeled
dataset_macho_unl = get_windows(dataset_macho_unl, window_size=window_size, sampling=sampling)

# Macho labeled
dataset_macho_lab = get_windows(dataset_macho_lab, window_size=window_size, sampling=sampling)

# Atlas
dataset_atlas = get_windows(dataset_atlas, window_size=window_size, sampling=sampling)

# Ogle
dataset_ogle = get_windows(dataset_ogle, window_size=window_size, sampling=sampling)

# Kepler
dataset_kepler = get_windows(dataset_kepler, window_size=window_size, sampling=sampling)
dataset_kepler_alcock = get_windows(dataset_kepler_alcock, window_size=window_size, sampling=sampling)
dataset_kepler_atlas = get_windows(dataset_kepler_atlas, window_size=window_size, sampling=sampling)
dataset_kepler_ogle = get_windows(dataset_kepler_ogle, window_size=window_size, sampling=sampling)

In [10]:
def get_days_by_windows(dataset):
    days_by_windows = []
    for i, lc_info in enumerate(dataset):
        lc_data = lc_info['input']
        try:
            days_by_windows.append(lc_data[-1][0] - lc_data[0][0].numpy())
        except:
            print(lc_data.shape)

        if i > 50000:
            break

    return days_by_windows

In [11]:
macho_unl_days_windows = get_days_by_windows(dataset_macho_unl)
macho_lab_days_windows = get_days_by_windows(dataset_macho_lab)
atlas_days_windows = get_days_by_windows(dataset_atlas)
ogle_days_windows = get_days_by_windows(dataset_ogle)
kepler_days_windows = get_days_by_windows(dataset_kepler)
kepler_alcock_days_windows = get_days_by_windows(dataset_kepler_alcock)
kepler_atlas_days_windows = get_days_by_windows(dataset_kepler_atlas)
kepler_ogle_days_windows = get_days_by_windows(dataset_kepler_ogle)

2023-11-06 05:11:15.529465: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at strided_slice_op.cc:117 : INVALID_ARGUMENT: slice index -1 of dimension 0 out of bounds.


(0, 3)


In [12]:
def print_stats(days_by_windows, name_dataset):
    print('{}'.format(name_dataset.upper()))
    print('- Median: {:.2f}'.format(np.median(days_by_windows)))
    print('- Mean: {:.2f}'.format(np.mean(days_by_windows)))
    print('- Std: {:.2f}\n'.format(np.std(days_by_windows)))

In [13]:
print('Number of days saved in each generated window:\n')
print_stats(macho_unl_days_windows, 'Macho Unlabeled')
print_stats(macho_lab_days_windows, 'Macho Labeled')
print_stats(atlas_days_windows, 'Atlas')
print_stats(ogle_days_windows, 'Ogle')
print_stats(kepler_days_windows, 'Kepler')
print_stats(kepler_alcock_days_windows, 'Kepler-Alcock')
print_stats(kepler_atlas_days_windows, 'Kepler-Atlas')
print_stats(kepler_ogle_days_windows, 'Kepler-Ogle')

Number of days saved in each generated window:

MACHO UNLABELED
- Median: 658.14
- Mean: 724.77
- Std: 555.84

MACHO LABELED
- Median: 678.42
- Mean: 801.60
- Std: 483.85

ATLAS
- Median: 577.70
- Mean: 599.85
- Std: 87.95

OGLE
- Median: 775.92
- Mean: 752.82
- Std: 508.14

KEPLER
- Median: 4.11
- Mean: 4.92
- Std: 7.96

KEPLER-ALCOCK
- Median: 801.27
- Mean: 815.16
- Std: 136.71

KEPLER-ATLAS
- Median: 893.68
- Mean: 909.47
- Std: 235.76

KEPLER-OGLE
- Median: 708.06
- Mean: 727.16
- Std: 181.40



In [14]:
print('MACHO labeled: {}'.format(np.median(macho_unl_days_windows) / np.median(macho_lab_days_windows)))
print('ATLAS: {}'.format(np.median(macho_unl_days_windows) / np.median(atlas_days_windows)))
print('OGLE: {}'.format(np.median(macho_unl_days_windows) / np.median(ogle_days_windows)))
print('KEPLER: {}'.format(np.median(macho_unl_days_windows) / np.median(kepler_days_windows)))
print('KEPLER-ALCOCK: {}'.format(np.median(macho_unl_days_windows) / np.median(kepler_alcock_days_windows)))
print('KEPLER-ATLAS : {}'.format(np.median(macho_unl_days_windows) / np.median(kepler_atlas_days_windows)))
print('KEPLER-OGLE: {}'.format(np.median(macho_unl_days_windows) / np.median(kepler_ogle_days_windows)))

MACHO labeled: 0.9701078534126282
ATLAS: 1.139237642288208
OGLE: 0.8482018709182739
KEPLER: 160.15493774414062
KEPLER-ALCOCK: 0.821365475654602
KEPLER-ATLAS : 0.7364314794540405
KEPLER-OGLE: 0.9294870495796204


In [15]:
print('MACHO labeled: {:.2f}'.format(np.mean(macho_unl_days_windows) / np.mean(macho_lab_days_windows)))
print('ATLAS: {:.2f}'.format(np.mean(macho_unl_days_windows) / np.mean(atlas_days_windows)))
print('OGLE: {:.2f}'.format(np.mean(macho_unl_days_windows) / np.mean(ogle_days_windows)))
print('KEPLER: {:.2f}'.format(np.mean(macho_unl_days_windows) / np.mean(kepler_days_windows)))
print('KEPLER-ALCOCK: {:.2f}'.format(np.mean(macho_unl_days_windows) / np.mean(kepler_alcock_days_windows)))
print('KEPLER-ATLAS : {:.2f}'.format(np.mean(macho_unl_days_windows) / np.mean(kepler_atlas_days_windows)))
print('KEPLER-OGLE: {:.2f}'.format(np.mean(macho_unl_days_windows) / np.mean(kepler_ogle_days_windows)))

MACHO labeled: 0.90
ATLAS: 1.21
OGLE: 0.96
KEPLER: 147.35
KEPLER-ALCOCK: 0.89
KEPLER-ATLAS : 0.80
KEPLER-OGLE: 1.00


In [3]:
''' 

Testing the pretrained model in other datasets 

'''

from src.models.astromer_1 import get_ASTROMER, test_step
from src.data import load_data
from src.data.zero import pretraining_pipeline

import toml
import os


def merge_metrics(**kwargs):
	merged = {}
	for key, value in kwargs.items():
		for subkey, subvalue in value.items():
			merged['{}_{}'.format(key, subkey)] = subvalue
	return merged


def average_logs(logs):
	N = len(logs)
	average_dict = {}
	for key in logs[0].keys():
		sum_log = sum(log[key] for log in logs)
		average_dict[key] = float(sum_log/N)
	return average_dict

###########################################################

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

#ds_names = ['alcock', 'ogle', 'atlas']
#ds_names = ['kepler', 'kepler_alcock_linear', 'kepler_atlas_linear', 'kepler_ogle_linear']
#ds_names = ['alcock', 'ogle', 'atlas', 'kepler', 'kepler_alcock_linear', 'kepler_atlas_linear', 'kepler_ogle_linear']
ds_names = ['kepler']

folds = [0, 1, 2]
spc_list = [50]
#spc_list = ['all']

ROOT = './presentation/experiments/astromer_1_pe'
pt_folder = 'results/pretraining/P02R01/pretrained_weights'

test_step_fn = test_step

with open('{}/{}/config.toml'.format(ROOT, pt_folder), mode="r") as fp:
    config = toml.load(fp)

with open('{}/{}/pe_config.toml'.format(ROOT, pt_folder), mode="r") as fp:
    pe_config = toml.load(fp)

astromer = get_ASTROMER(num_layers=config['num_layers'],
                        num_heads=config['num_heads'],
                        head_dim=config['head_dim'],
                        mixer_size=config['mixer'],
                        dropout=config['dropout'],
                        pe_type=config['pe_type'],
                        pe_config=pe_config,
                        window_size=config['window_size'],
                        encoder_mode=config['encoder_mode'],
                        average_layers=config['avg_layers'])


2023-10-27 01:13:58.734749: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Using masking...
Using masking...


In [6]:
astromer.get_layer('encoder').data_name = 'marti'

In [7]:
astromer.get_layer('encoder').data_name

'marti'

In [8]:
astromer.get_layer('encoder').data_name= 'perra'

In [10]:
astromer.get_layer('encoder').data_name

'perra'